<a href="https://colab.research.google.com/github/Yechang618/SLoG-Net-Example/blob/main/SLoG_Net_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# !pip install hdf5storage
# !pip install ipdb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !cp -r drive/MyDrive/slog-net/SLOGmodules .
!cp -r drive/MyDrive/slog-net/SLoGexample .

In [3]:
# !cp -r drive/MyDrive/slog-net/SLoGexample .
import numpy as np
from numpy import linalg as LA
import torch; torch.set_default_dtype(torch.float64)
import torch.nn as nn
import torch.optim as optim
import copy
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy import linalg
from timeit import default_timer as timer
import networkx as nx
import os
import pickle
import datetime
#### Import SLOG packakes
from SLoGexample import SLOGexperiment as SLOGexperiment
import torch
import os

In [4]:
useGPU = True
if useGPU and torch.cuda.is_available():
    device = 'cuda:0'
    torch.cuda.empty_cache()
else:
    device = 'cpu'
# Notify:
device = torch.device(device)
print("Device selected: %s" % device)

Device selected: cpu


Parameters

In [5]:
### Model parameters
K = 5 # number of layers

filterTrainType = 'h'
### Simulation parameters
simuParas = {}
nTrain_slog = 20000
P = 100
batchsize_slog = P
# nValid = batchsize_slog
# nTest = batchsize_slog
nValid = P
nTest = P
nEpochs = 20
# model_number = 0
model_number = 1 # SLOG-Net-v3

early_stop_criteria = 1e-5
### Data parameters
L = 5
alpha = 1.0

### Graph parameters

nNodes = 20 # Number of nodes
nClasses = 3 # Number of classes (i.e. number of communities)
N_C = int(0.15*nNodes) # Number of sources per signal
S = N_C
C = nNodes # constrain constant
q = 2 # Number of constrain vectors for SLOG-Net-v3
simuParas['nNodes'] = nNodes
simuParas['nClasses'] = nClasses
simuParas['N_C'] = N_C
simuParas['S'] = S
simuParas['model_number'] = model_number

graphType = 'ER' # Type of graph
graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
graphOptions['probIntra'] = 0.3 # Probability of drawing edges


## Filter type: g or h
filterType = 'h'
## Signal mode: gaussion or 1
signalMode = 'Gaussion'
## Train mode: Wt or default (Vdiag(\tilde{h})V^T)
trainMode = 'default'
## Filter mode: Wt or default (Vdiag(\tilde{h})V^T)
filterMode = 'default'

## Selection mode: random or nodes with top-N_C degree
selectMode = 'random'

## Noise level
noiseLevel = 0.0
noiseType = 'uniform'

simuParas['nTrain_slog'] = nTrain_slog
simuParas['batchsize_slog'] = batchsize_slog
simuParas['nValid'] = nValid
simuParas['nTest'] = nTest
simuParas['L'] = L
simuParas['noiseLevel'] = noiseLevel
simuParas['noiseType'] = noiseType
simuParas['filterType'] = filterType
simuParas['signalMode'] = signalMode
simuParas['trainMode'] = trainMode
simuParas['filterMode'] = filterMode
simuParas['selectMode'] = selectMode
simuParas['graphType'] = graphType
simuParas['alpha'] = alpha
simuParas['nEpochs'] = nEpochs
simuParas['device'] = device
simuParas['early_stop_criteria'] = early_stop_criteria
# simuParas['useZtoTrain'] = False

tMax = None # Maximum number of diffusion times (W^t for t < tMax)
tMax = 5
simuParas['tMax'] = tMax

## model parameters
modelParas = {}
modelParas['filterTrainType'] = filterTrainType
modelParas['C'] = C
modelParas['q'] = q
modelParas['K'] = K

## Experiment parameters
expParas = {}
expParas['nRealiz'] = 2



Experiment: ER graph



In [6]:
## q = 1
graphType = 'ER' # Type of graph
graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
graphOptions['probIntra'] = 0.3 # Probability of drawing edges

simuParas['graphType'] = graphType
simuParas['nNodes'] = 20
expParas['nRealiz'] = 2
simuParas['nEpochs'] = 20
modelParas['q'] = 2
P = 400
batchsize_slog = P
nValid = P
nTest = P
simuParas['batchsize_slog'] = batchsize_slog
simuParas['nValid'] = nValid
simuParas['nTest'] = nTest
simuParas['nTrain_slog'] = 20000
simuParas['alpha'] = 1.0
simuParas['batchsize_slog'] = P
simuParas['nValid'] = P
simuParas['nTest'] = P
simuParas['L'] = 3
simuParas['model_number'] = 1
result_1 = SLOGexperiment.slog_experiments(simuParas = simuParas,
                 graphOptions = graphOptions,
                 modelParas = modelParas, expParas = expParas)




    (E:  9, B:  41) 
	 Loss: 0.0024 [T] 0.0030 [V] 	 RE: 0.1316 [T] 	 RE: 0.1401 [V] 	 Best Loss: 0.0030 [V] 	 Loss: 0.2160 [Test]
Best model updated at: 8 40
rho_1 tensor([0.0000, 0.8506, 0.9916, 2.2244, 1.8448])
eta_1 tensor([2.3505, 1.5708, 0.4146, 1.2295, 1.9646])
lmbd tensor([0.0645, 0.0932, 0.0442, 0.4441, 0.8896])
alpha_1 tensor([-0.1851, -1.2281,  0.9987,  0.4019, -0.8680])
alpha_2 tensor([ 1.2665,  0.3453, -1.1929,  0.2273, -2.0421])
beta_1 tensor([-0.3005, -0.1704, -0.1644, -0.6201,  0.9825])
beta_2 tensor([-0.0049, -1.7711, -1.5386, -0.9645,  0.8489])
beta_3 tensor([-1.0445, -0.0845, -0.9689,  0.8180, -0.0206])
gamma_1 tensor([ 0.0111, -1.5353,  1.4190,  0.1328, -0.9634])
M tensor([[[-0.0634, -2.0192,  1.2942, -0.0874, -0.2755],
         [ 1.3772,  0.1854, -0.9061,  0.3659, -1.7133]],

        [[-0.1140, -1.8049,  0.7399, -0.1507,  0.3608],
         [-2.8877,  1.3840, -0.4954,  0.0820, -1.1535]],

        [[-2.4420,  1.0770,  2.4429, -0.6419,  0.3403],
         [ 2.5695,  0

In [16]:
experiment_results = result_1.experiment_results
for exp_result in experiment_results:
  print(exp_result['model'])
  print(exp_result['Graph'])
  print(exp_result['training result'])


Name: SLOG-Net
Number of learnable parameters: 255

Model architecture:
----- -------------

GraphSLoG_v3()

Loss function:
---- ---------

<function myLoss at 0x79fba0a60f70>

Optimizer:
----------

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)
Training algorithm:
-------- ----------

Evaluation algorithm:
---------- ----------

<function evaluate at 0x79fb9da76dd0>

{'bestModel': GraphSLoG_v3(), 'minLossValid': 0.0006853591081975277, 'minLossTrain': 0.0003501240287375447}
Name: SLOG-Net
Number of learnable parameters: 255

Model architecture:
----- -------------

GraphSLoG_v3()

Loss function:
---- ---------

<function myLoss at 0x79fba0a60f70>

Optimizer:
----------

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: Non

In [ ]:
simuParas = {}
### Graph parameters


simuParas['nNodes'] = nNodes
simuParas['nClasses'] = nClasses
simuParas['N_C'] = N_C



# graphType = 'BA' # Type of graph
# graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
# graphOptions['alpha'] = 1.0

# graphType = 'ER' # Type of graph
# graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
# graphOptions['probIntra'] = 0.3 # Probability of drawing edges

simuParas['graphType'] = graphType
simuParas['graphOptions'] = graphOptions



simuParas['nTrain_slog'] = nTrain_slog
simuParas['batchsize_slog'] = batchsize_slog
simuParas['nValid'] = nValid
simuParas['nTest'] = nTest
simuParas['L'] = L
simuParas['noiseLevel'] = noiseLevel
simuParas['noiseType'] = noiseType
simuParas['filterType'] = filterType
simuParas['signalMode'] = signalMode
simuParas['trainMode'] = trainMode
simuParas['filterMode'] = filterMode
simuParas['selectMode'] = selectMode
simuParas['graphType'] = graphType
simuParas['alpha'] = alpha
simuParas['nEpochs'] = nEpochs
simuParas['C'] =  C
simuParas['K'] =  K
simuParas['N_realiz'] =  N_realiz
simuParas['device'] =  device
simuParas['model_number'] = model_number
modelParas = {}
modelParas['q'] = q
modelParas['device'] = device
simuParas['supp_thres'] = 0.1
# if device == 'cuda:0':
#   simuParas['useZtoTrain'] = True
# else:
#   simuParas['useZtoTrain'] = False

simuParas['useZtoTrain'] = True


tMax = None # Maximum number of diffusion times (W^t for t < tMax)
tMax = 5
simuParas['tMax'] = tMax



today = datetime.datetime.now().strftime("%m%d%H%M")
print_date = today
print('Print date: ', print_date)



In [ ]:
# #### Test 1
# ### SLOG-Net only, v.s. noise level
# save_test_result = True
# N_realiz = 100
# simuParas['N_realiz'] =  N_realiz

# N_model = len(modelDirList)
# noiseLvs = n0 + d_noiseLvs*np.arange(N_noiseLvs)
# result_exp_rex = np.zeros([N_noiseLvs, N_model,N_realiz])
# result_exp_acc = np.zeros([N_noiseLvs, N_model,N_realiz])
# result_exp_reg = np.zeros([N_noiseLvs, N_model,N_realiz])
# result_elapse_slog = np.zeros([N_noiseLvs, N_model,N_realiz])
# n_model = 0
# for modelDir in modelDirList:
#     for n_nlvs in range(N_noiseLvs):
#         simuParas['noiseLevel'] = noiseLvs[n_nlvs]
#         print('Model ', n_model, ', in ', modelDir,',noiseLevel = ', noiseLvs[n_nlvs])
#         result = SLOGnoiseTest.noiseTest_gdrive(nNodes,P,S, modelDir, simuParas = simuParas,modelParas = modelParas, location = location , normalize_g_hat = normalize_g_hat)
#         result_exp_rex[n_nlvs,n_model,:] = result['re_x']
#         result_exp_acc[n_nlvs,n_model,:] = result['acc_x']
#         result_exp_reg[n_nlvs,n_model,:] = result['re_g']
#         result_elapse_slog[n_nlvs,n_model,:] = result['elapse_slog']
#     n_model += 1
# test_result = {}
# test_result['result_exp_rex'] = result_exp_rex
# test_result['result_exp_acc'] = result_exp_acc
# test_result['result_exp_reg'] = result_exp_reg
# test_result['result_elapse_slog'] = result_elapse_slog
# test_info = {}
# test_info['nTrain'] = nTrain
# test_info['graphType'] = graphType
# test_info['list_name'] = list_name
# test_result['test_info'] = test_info

# saveDir_testResult = os.path.join(saveDir_dropbox_root,'testResult')
# testResult_saveName = "testResult_n" + str(N_realiz) + '_' + print_date+ "_gt" + str(graphType) + "_nTrain" + str(nTrain) + "_" + list_name + ".pt"

# if not os.path.exists(saveDir_testResult):
#   os.makedirs(saveDir_testResult)

# saveDir_testResult_fileName = os.path.join(saveDir_testResult,testResult_saveName)

# if save_test_result:
#   torch.save(test_result,saveDir_testResult_fileName)
#   print('data_graph_name: ', testResult_saveName)